In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import numpy as np
import pandas as pd

from gensim.models import Word2Vec
from scipy import spatial

from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import RegexpTokenizer
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [28]:
df = pd.read_csv('/content/drive/My Drive/Hate_speech/train_tweets.csv')

In [29]:
df

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [30]:
#check for null values
df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [31]:
#removing links
df['clean_text'] = df['tweet'].apply(lambda s: ' '.join(re.sub("(w+://S+)", " ", s).split()))

In [32]:
df[['tweet', 'clean_text']].iloc[31954]

tweet         #holiday   bull up: you will dominate your bul...
clean_text    #holiday bull up: you will dominate your bull ...
Name: 31954, dtype: object

In [33]:
import pandas as pd
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop = set(stopwords.words('english'))

def rem_stopwords(input_text):
    words=input_text.lower().split()
    return ' '.join([word for word in words if word not in stop])

df['clean_text'] = df['tweet'].apply(lambda s: rem_stopwords(s))
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,@user father dysfunctional selfish drags kids ...
1,2,0,@user @user thanks for #lyft credit i can't us...,@user @user thanks #lyft credit can't use caus...
2,3,0,bihday your majesty,bihday majesty
3,4,0,#model i love u take with u all the time in ...,#model love u take u time urð±!!! ððð...
4,5,0,factsguide: society now #motivation,factsguide: society #motivation
...,...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...,ate @user isz youuu?ððððððð...
31958,31959,0,to see nina turner on the airwaves trying to...,see nina turner airwaves trying wrap mantle ge...
31959,31960,0,listening to sad songs on a monday morning otw...,listening sad songs monday morning otw work sad
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...","@user #sikh #temple vandalised #calgary, #wso ..."


In [34]:
#removing punctuation
df['clean_text'] = df['clean_text'].apply(lambda s: ' ' .join(re.sub("[.,!?:;-='...'""'@#_]", " ", s).split()))

In [36]:
df[['tweet', 'clean_text']]

,tweet,clean_text
0,@user when a father is dysfunctional and is s...,user father dysfunctional selfish drags kids d...
1,@user @user thanks for #lyft credit i can't us...,user user thanks lyft credit can t use cause o...
2,bihday your majesty,bihday majesty
3,#model i love u take with u all the time in ...,model love u take u time urð± ðððð...
4,factsguide: society now #motivation,factsguide society motivation
...,...,...
31957,ate @user isz that youuu?ðððððð...,ate user isz youuu ððððððð...
31958,to see nina turner on the airwaves trying to...,see nina turner airwaves trying wrap mantle ge...
31959,listening to sad songs on a monday morning otw...,listening sad songs monday morning otw work sad
31960,"@user #sikh #temple vandalised in in #calgary,...",user sikh temple vandalised calgary wso condem...


In [10]:
#remove numbers
df['clean_text'].replace('d+', '', regex=True, inplace=True)
df[['tweet', 'clean_text']]

,tweet,clean_text
0,@user when a father is dysfunctional and is s...,user when a father is ysfunctional an is so se...
1,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft creit i can t use ca...
2,bihday your majesty,bihay your majesty
3,#model i love u take with u all the time in ...,moel i love u take with u all the time in urð...
4,factsguide: society now #motivation,factsguie society now motivation
...,...,...
31957,ate @user isz that youuu?ðððððð...,ate user isz that youuu ðððððð...
31958,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to w...
31959,listening to sad songs on a monday morning otw...,listening to sa songs on a monay morning otw t...
31960,"@user #sikh #temple vandalised in in #calgary,...",user sikh temple vanalise in in calgary wso co...


In [37]:
#Removing emojis
def remove_emojis(inputString):
  return inputString.encode('ascii', 'ignore').decode('ascii')


In [38]:
df['clean_text'] = df['clean_text'].apply(lambda s: remove_emojis(s))

In [39]:
df[['tweet',  'clean_text']]

,tweet,clean_text
0,@user when a father is dysfunctional and is s...,user father dysfunctional selfish drags kids d...
1,@user @user thanks for #lyft credit i can't us...,user user thanks lyft credit can t use cause o...
2,bihday your majesty,bihday majesty
3,#model i love u take with u all the time in ...,model love u take u time ur
4,factsguide: society now #motivation,factsguide society motivation
...,...,...
31957,ate @user isz that youuu?ðððððð...,ate user isz youuu
31958,to see nina turner on the airwaves trying to...,see nina turner airwaves trying wrap mantle ge...
31959,listening to sad songs on a monday morning otw...,listening sad songs monday morning otw work sad
31960,"@user #sikh #temple vandalised in in #calgary,...",user sikh temple vandalised calgary wso condem...


In [ ]:
'''def rem_stopwords(input_text):
  words = input_text.lower().split()
  noise_free_words = [word for word in words if word not in stop]
  noise_free_text = " ".join(noise_free_words)
  return noise_free_text
  '''

In [ ]:
'''df['clean_text'] = df['tweet'].apply(lambda s: rem_stopwords(s))
df[['tweet', 'clean_text']]
'''

NameError: ignored

In [41]:
from nltk import word_tokenize
nltk.download('punkt')

df['clean_text'] = df['clean_text'].apply(word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,"[user, father, ysfunctional, selfish, rags, ki..."
1,2,0,@user @user thanks for #lyft credit i can't us...,"[user, user, thanks, lyft, creit, use, cause, ..."
2,3,0,bihday your majesty,"[bihay, majesty]"
3,4,0,#model i love u take with u all the time in ...,"[moel, love, u, take, u, time, ur]"
4,5,0,factsguide: society now #motivation,"[factsguie, society, motivation]"
...,...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...,"[ate, user, isz, youuu]"
31958,31959,0,to see nina turner on the airwaves trying to...,"[see, nina, turner, airwaves, trying, wrap, ma..."
31959,31960,0,listening to sad songs on a monday morning otw...,"[listening, sa, songs, monay, morning, otw, wo..."
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...","[user, sikh, temple, vanalise, calgary, wso, c..."


In [45]:
#Lemmatization

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatiser = WordNetLemmatizer()
df['clean_text'] = df['clean_text'].apply(lambda tokens: [lemmatiser.lemmatize(token, pos='v') for token in tokens])

df[['tweet', 'clean_text']]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,tweet,clean_text
0,@user when a father is dysfunctional and is s...,"[user, father, dysfunctional, selfish, drag, k..."
1,@user @user thanks for #lyft credit i can't us...,"[user, user, thank, lyft, credit, can, t, use,..."
2,bihday your majesty,"[bihday, majesty]"
3,#model i love u take with u all the time in ...,"[model, love, u, take, u, time, ur]"
4,factsguide: society now #motivation,"[factsguide, society, motivation]"
...,...,...
31957,ate @user isz that youuu?ðððððð...,"[eat, user, isz, youuu]"
31958,to see nina turner on the airwaves trying to...,"[see, nina, turner, airwaves, try, wrap, mantl..."
31959,listening to sad songs on a monday morning otw...,"[listen, sad, songs, monday, morning, otw, wor..."
31960,"@user #sikh #temple vandalised in in #calgary,...","[user, sikh, temple, vandalise, calgary, wso, ..."
